[1] 모듈 로딩

In [ ]:
from urllib.request import urlopen  
from bs4 import BeautifulSoup
import pandas as pd

[2] 웹에서 데이터 다운로드

In [ ]:
## 다운로드 받을 웹 사이트 URL
W_URL = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4'

## 다운로딩 
movie = urlopen(W_URL)

## 로우데이터만 가져오기 => byte 타입의 데이터
m_data = movie.read()

[3] 웹에서 원하는 데이터 추출 : 스크래핑

In [ ]:
## BS객체 생성 : 마크업언어 스타일 데이터 ===> 사람이해하기 쉽게 객체화 
bs=BeautifulSoup(m_data, 'html.parser')

In [ ]:
links = bs.find_all('a', class_='inner')

movie_links = []
for a in links[:15]:
    movie_links.append("https://search.naver.com/search.naver" + a.get("href"))

movie_links

['https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=37041608&qvt=0&query=%EC%9C%84%ED%82%A4%EB%93%9C%3A%20%ED%8F%AC%20%EA%B5%BF',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=2716530&qvt=0&query=%EB%82%98%EC%9A%B0%20%EC%9C%A0%20%EC%94%A8%20%EB%AF%B8%203',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=37363629&qvt=0&query=%EA%B7%B9%EC%9E%A5%ED%8C%90%20%EC%B2%B4%EC%9D%B8%EC%86%8C%20%EB%A7%A8%3A%20%EB%A0%88%EC%A0%9C%ED%8E%B8',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=37520907&qvt=0&query=%EA%B5%AD%EB%B3%B4',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=36831199&qvt=0&query=%ED%94%84%EB%A0%88%EB%8D%B0%ED%84%B0%3A%20%EC%A3%BD%EC%9D%8C%EC%9D%98%20%EB%95%85',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=35072207&qvt=0&query=%EB%82%98%ED%98%BC%EC%9E%90%20%

In [ ]:
list = []

for movie_url in movie_links:
    ## 다운로드 받을 웹 사이트 URL
    M_URL = movie_url
    ## 다운로딩 
    Movie = urlopen(M_URL)
    ## 로우데이터만 가져오기 => byte 타입
    M_data = Movie.read()

    ## BS객체 생성 : 마크업언어 => 사람이해하기 쉽게 객체화 
    bs=BeautifulSoup(M_data, 'html.parser')

    # 영화제목
    title = bs.find('strong', class_='_text').string

    # 영화장르
    janre = bs.find('div', class_='info_group').find('dd').contents[0].strip()

    # 국가
    country = bs.find('div', class_='info_group').find('dd').contents[2].strip()
    if country[0].isdigit():
        country = "정보없음"

    # 순위
    rank = bs.find('span', class_='normal_text').find('em').string

    ## 평점
    grade = bs.find('span', class_='this_text_bold').string

    list.append([title, janre, country, rank, grade])

DF = pd.DataFrame(list, columns=['영화제목', '장르', '국가', '순위', '평점'])

DF.set_index('순위')

                 영화제목     장르    국가    평점
순위                                      
1            위키드: 포 굿    판타지  정보없음  7.54
2          나우 유 씨 미 3     범죄    미국  7.83
3      극장판 체인소 맨: 레제편  애니메이션    일본  9.27
4                  국보    드라마  정보없음  9.10
5         프레데터: 죽음의 땅     SF    미국  8.62
6             나혼자 프린스    코미디  대한민국  7.53
7             퍼스트 라이드    코미디  대한민국  7.57
8              세계의 주인    드라마  대한민국  9.08
9    극장판 귀멸의 칼날: 무한성편  애니메이션    일본  9.03
10  극장판 똘똘이: 아기공룡의 비밀  애니메이션  정보없음  8.14
11              8번 출구    스릴러    일본  8.36
12               코렐라인  애니메이션    미국  8.46
13        엔드 오브 에반게리온  애니메이션    일본  9.26
14                가타카     SF    미국  9.10
15    극장판 주술회전: 회옥·옥절  애니메이션  정보없음  7.98


In [ ]:
DF['순위'].corr(DF['평점'])

# 평점과 박스오피스 순위는 서로 관계 없음.(0.24)

np.float64(0.24185904313695056)

In [ ]:
DF['국가'].value_counts()

# 미국, 일본 영화가 많음.

국가
정보없음    4
미국      4
일본      4
대한민국    3
Name: count, dtype: int64

In [ ]:
DF['장르'].value_counts()

## 애니메이션이 가장 많음


장르
애니메이션    6
드라마      2
SF       2
코미디      2
판타지      1
범죄       1
스릴러      1
Name: count, dtype: int64